In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

REPO_NAME = 'draft-assistant'
CWD = str(os.getcwd())
REPO_DIR = CWD[:CWD.find(REPO_NAME)+len(REPO_NAME)]
sys.path.insert(0,REPO_DIR)

In [ ]:
from src.league.league import League

l = League(
    draft_position=12,
    n_teams=12
)